# anomaly_dashboard.ipynb
### Interactive Anomaly Dashboard for SQL Logs
**Author**: Dr. William Kandolo  
**Purpose**: Visual exploration of anomaly scores and ground truth labels
**Input**: sql_train.csv (with 'anomaly_score' and 'anomaly' columns)

In [ ]:
# STEP 1: IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from IPython.display import display
import ipywidgets as widgets

In [ ]:
# STEP 2: LOAD DATA
df = pd.read_csv('datasets/sql_train.csv')

In [ ]:
# STEP 3: CHECK REQUIRED COLUMNS
required_cols = ['timestamp', 'anomaly_score', 'anomaly']
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Missing column: {col}")
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp'])

In [ ]:
# STEP 4: OVERVIEW STATISTICS
print("Data shape:", df.shape)
print("Anomaly rate:", df['anomaly'].mean())

In [ ]:
# STEP 5: TIME SERIES PLOT
fig1 = px.line(df, x='timestamp', y='anomaly_score',
               color=df['anomaly'].map({0: 'Normal', 1: 'Anomaly'}),
               title='Anomaly Score Over Time')
fig1.update_layout(legend_title='Event Type')
fig1.show()

In [ ]:
# STEP 6: HISTOGRAM OF ANOMALY SCORES
fig2 = px.histogram(df, x='anomaly_score', nbins=50,
                    color=df['anomaly'].map({0: 'Normal', 1: 'Anomaly'}),
                    title='Distribution of Anomaly Scores')
fig2.show()

In [ ]:
# STEP 7: FEATURE HEATMAP
feature_cols = df.select_dtypes(include=[np.number]).drop(columns=['anomaly_score', 'anomaly']).columns
plt.figure(figsize=(12, 6))
sns.heatmap(df[feature_cols].corr(), cmap='coolwarm', annot=False)
plt.title("Feature Correlation Heatmap")
plt.tight_layout()
plt.show()

In [ ]:
# STEP 8: INTERACTIVE SLIDER
score_slider = widgets.FloatRangeSlider(
    value=[0.2, 0.8],
    min=0.0,
    max=1.0,
    step=0.01,
    description='Score range:',
    continuous_update=False
)

def filter_and_show(range_vals):
    low, high = range_vals
    filtered = df[(df['anomaly_score'] >= low) & (df['anomaly_score'] <= high)]
    display(filtered[['timestamp', 'anomaly_score', 'anomaly']].head(20))

widgets.interact(filter_and_show, range_vals=score_slider)